In [1]:
import numpy as np

In [136]:
## RUN testing_didgrad_and_locate.f and compare outputs with this notebook

from numpy import exp, zeros, where, sqrt, cumsum , pi, outer, sinh, cosh, min, dot, array,log10

def did_grad_cp( t, p, t_table, p_table, grad, cp, calc_type):
    """
    Parameters
    ----------
    t : float
        Temperature  value
    p : float 
        Pressure value
    t_table : array 
        array of Pressure values with 53 entries
    p_table : array 
        array of Pressure value with 26 entries
    grad : array 
        array of gradients of dimension 53*26
    cp : array 
        array of cp of dimension 53*26
    calc_type : int 
        if 0 will return both gradx,cpx , if 1 will return only gradx, if 2 will return only cpx
    
    Returns
    -------
    float 
        if calc_type= 0 grad_x,cp_x
        if calc_type= 1 grad_x 
        if calc_type= 2 cp_x
    
    """
    # Python version of DIDGRAD function in convec.f in EGP
    # This has been benchmarked with the fortran version
    
       
    temp_log= log10(t)
    pres_log= log10(p)
    
    pos_t = locate(t_table, temp_log)
    pos_p = locate(p_table, pres_log)

    ipflag=0
    if pos_p ==0: ## lowest pressure point
        factkp= 0.0
        ipflag=1
    elif pos_p ==25 : ## highest pressure point
        factkp= 1.0
        pos_p=24  ## use highest point
        ipflag=1

    itflag=0
    if pos_t ==0: ## lowest pressure point
        factkt= 0.0
        itflag=1
    elif pos_t == 52 : ## highest temp point
        factkt= 1.0
        pos_t=51 ## use highest point
        itflag=1
    
    if (pos_p > 0) and (pos_p < 26) and (ipflag == 0):
        factkp= (-p_table[pos_p]+pres_log)/(p_table[pos_p+1]-p_table[pos_p])
    
    if (pos_t > 0) and (pos_t < 53) and (itflag == 0):
        factkt= (-t_table[pos_t]+temp_log)/(t_table[pos_t+1]-t_table[pos_t])

    
    gp1 = grad[pos_t,pos_p]
    gp2 = grad[pos_t+1,pos_p]
    gp3 = grad[pos_t+1,pos_p+1]
    gp4 = grad[pos_t,pos_p+1]

    cp1 = cp[pos_t,pos_p]
    cp2 = cp[pos_t+1,pos_p]
    cp3 = cp[pos_t+1,pos_p+1]
    cp4 = cp[pos_t,pos_p+1]


    

    grad_x = (1.0-factkt)*(1.0-factkp)*gp1 + factkt*(1.0-factkp)*gp2 + factkt*factkp*gp3 + (1.0-factkt)*factkp*gp4
    cp_x= (1.0-factkt)*(1.0-factkp)*cp1 + factkt*(1.0-factkp)*cp2 + factkt*factkp*cp3 + (1.0-factkt)*factkp*cp4
    cp_x= 10**cp_x
    
    if calc_type == 0:
        return grad_x,cp_x
    elif calc_type == 1:
        return grad_x
    elif calc_type == 2 :
        return cp_x

def locate(array,value):
    """
    Parameters
    ----------
    array : array
        Array to be searched.
    value : float 
        Value to be searched for.
    
    
    Returns
    -------
    int 
        location of nearest point by bisection method 
    
    """
    
    
    n = len(array)
    
    
    jl = 0
    ju = n
    while (ju-jl > 1):
        jm=int(0.5*(ju+jl)) 
        if (value >= array[jm]):
            jl=jm
        else:
            ju=jm
    
    if (value <= array[0]): # if value lower than first point
        jl=0
    elif (value >= array[-1]): # if value higher than first point
        jl= n-1
    
    return jl

In [137]:
## creating arbitrary but same inputs for both fortran and python version


for i in range(53):
    t_table[i]=i
for i in range(26):
    p_table[i]=i



grad=np.zeros(shape=(53,26))
for i in range(53):
    for j in range(26):
        grad[i,j]=i+j
cp=grad/100.0

In [140]:
gradx,cpx=did_grad_cp(200,303, t_table, p_table, grad, cp, calc_type=0)

In [141]:
print(gradx) ## compare this with fortran output of the same input
## one can change both python and fortran inputs arbitrarily and get same results
## I have tried out of range values and still gives same results
## so consider this benchmarked with the fortran version of DIDGRAD and LOCATE

4.782472624166286


In [143]:
print(cpx)

1.116412591991876


In [146]:
cpx=did_grad_cp(200,303, t_table, p_table, grad, cp, calc_type=2)
print(cpx)

1.116412591991876
